In [ ]:
import json

def screen_bounds(json_data):
    bounds = json_data['bounds']
    left, top, right, bottom = bounds
    width = right - left
    height = bottom - top
    return width, height

def parse_annotations(json_data):
    objects = []
    
    def parse_children(children):
        for item in children:
            if 'bounds' in item:
                left, top, right, bottom = item['bounds']
                width = right - left
                height = bottom - top
                objects.append({
                    'left': left,
                    'top': top,
                    'right': right,
                    'bottom': bottom,
                    'width': width,
                    'height': height
                })
            if 'children' in item:
                parse_children(item['children'])

    parse_children(json_data['children'])
    return objects

def compute_balance_scores(objects):
    screen_width, screen_height = screen_bounds(json_data)
    left_area = right_area = top_area = bottom_area = 0
    left_distance = right_distance = top_distance = bottom_distance = 0
    
    if objects == []:
        return 0
    else:
      for obj in objects:
          left_area += obj['width']*obj['height'] if (obj['left']+obj['right'])/2 < screen_width/2 else 0
          right_area += obj['width']*obj['height'] if (obj['left']+obj['right'])/2  >= screen_width/2 else 0
          top_area += obj['width']*obj['height'] if (obj['top']+obj['bottom'])/2  < screen_height/2 else 0
          bottom_area += obj['width']*obj['height'] if (obj['top']+obj['bottom'])/2 >= screen_height/2 else 0
          
          left_distance += abs((obj['left'] + obj['width'])/2 - screen_width) if (obj['left']+obj['right'])/2 < screen_width/2 else 0
          right_distance += abs((obj['left'] + obj['width'])/2 - screen_width) if (obj['left']+obj['right'])/2 < screen_width/2 else 0
          top_distance += abs((obj['top'] + obj['height'])/2 - screen_height) if (obj['top']+obj['bottom'])/2  < screen_height/2 else 0
          bottom_distance += abs((obj['top'] + obj['height'])/2 - screen_height) if (obj['top']+obj['bottom'])/2  < screen_height/2 else 0
      
      left_weight = left_area / max(left_area, right_area)    
      right_weight = right_area / max(left_area, right_area)
      top_weight = top_area / max(top_area, bottom_area)
      bottom_weight = bottom_area / max(top_area, bottom_area)
      
      vertical_balance = abs(left_weight - right_weight)
      horizontal_balance = abs(top_weight - bottom_weight)
      
      balance_measure = 1 - (vertical_balance + horizontal_balance) / 2
    
    return balance_measure

def balance_score(json_data):
    objects = parse_annotations(json_data)
    balance_score = compute_balance_scores(objects)
    print("Balance Score:", balance_score)
    return balance_score

import os
for filename in os.listdir('semantic_annotations'):
    if filename.endswith(".json"):
      with open('semantic_annotations/' + filename) as f:
          json_data = json.load(f)
          balance_score(json_data)
    else:
        continue